## Step 0: Add Graphistry and/or Huggginface Secrets in Kaggle

In [16]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
graphistry.register(
    api=3,
    protocol="https",
    server="hub.graphistry.com",
    personal_key_id=user_secrets.get_secret("Graphistry_Personal_Key_ID"),
    personal_key_secret=user_secrets.get_secret("Graphistry_Personal_Secret_Key")
)

## Step 1: Verify Dual GPU Environment

In [1]:
import subprocess
import os

print("="*70)
print("🔍 SPLIT-GPU ENVIRONMENT CHECK")
print("="*70)

# Check GPUs
result = subprocess.run(
    ["nvidia-smi", "--query-gpu=index,name,memory.total,memory.free", "--format=csv,noheader"],
    capture_output=True, text=True
)

gpus = result.stdout.strip().split('\n')
print(f"\n📊 Detected {len(gpus)} GPU(s):")
for gpu in gpus:
    print(f"   {gpu}")

if len(gpus) >= 2:
    print("\n✅ Dual T4 ready for split-GPU operation!")
    print("   GPU 0 → llama-server (LLM)")
    print("   GPU 1 → RAPIDS/Graphistry")
else:
    print("\n⚠️ Need 2 GPUs for split operation")

🔍 SPLIT-GPU ENVIRONMENT CHECK

📊 Detected 2 GPU(s):
   0, Tesla T4, 15360 MiB, 15096 MiB
   1, Tesla T4, 15360 MiB, 15096 MiB

✅ Dual T4 ready for split-GPU operation!
   GPU 0 → llama-server (LLM)
   GPU 1 → RAPIDS/Graphistry


## Step 2: Install Dependencies

In [2]:
%%time
print("📦 Installing dependencies...")

# Install llcuda v2.2.0 (force fresh install to ensure correct binaries)
!pip install -q --no-cache-dir git+https://github.com/llcuda/llcuda.git@v2.2.0

# Install cuGraph (matching Kaggle RAPIDS 25.6.0)
!pip install -q --extra-index-url=https://pypi.nvidia.com "cugraph-cu12==25.6.*"

# Install Graphistry
!pip install -q graphistry

# Verify installations
import llcuda
print(f"\n✅ llcuda {llcuda.__version__} installed")

try:
    import cudf
    import cugraph
    print(f"✅ cuDF {cudf.__version__}")
    print(f"✅ cuGraph {cugraph.__version__}")
except ImportError as e:
    print(f"⚠️ RAPIDS: {e}")

try:
    import graphistry
    print(f"✅ Graphistry {graphistry.__version__}")
except ImportError as e:
    print(f"⚠️ Graphistry: {e}")

📦 Installing dependencies...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 42.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 46.3 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.22.1 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.4.2 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
bigframes 2.26.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.8/439.8 kB 10.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━


🎯 llcuda v2.2.0 First-Time Setup - Kaggle 2× T4 Multi-GPU

🎮 GPU Detected: Tesla T4 (Compute 7.5)
  ✅ Tesla T4 detected - Perfect for llcuda v2.1!
🌐 Platform: Colab

📦 Downloading Kaggle 2× T4 binaries (~961 MB)...
    Features: FlashAttention + Tensor Cores + Multi-GPU tensor-split

➡️  Attempt 1: HuggingFace (llcuda-v2.2.0-cuda12-kaggle-t4x2.tar.gz)
📥 Downloading v2.2.0 from HuggingFace Hub...
   Repo: waqasm86/llcuda-binaries
   File: v2.2.0/llcuda-v2.2.0-cuda12-kaggle-t4x2.tar.gz


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


v2.2.0/llcuda-v2.2.0-cuda12-kaggle-t4x2.(…):   0%|          | 0.00/1.01G [00:00<?, ?B/s]

🔐 Verifying SHA256 checksum...
   ✅ Checksum verified
📦 Extracting llcuda-v2.2.0-cuda12-kaggle-t4x2.tar.gz...
Found 21 files in archive
Extracted 21 files to /root/.cache/llcuda/extract_2.2.0
✅ Extraction complete!
  Found bin/ and lib/ under /root/.cache/llcuda/extract_2.2.0/llcuda-v2.2.0-cuda12-kaggle-t4x2
  Copied 13 binaries to /usr/local/lib/python3.12/dist-packages/llcuda/binaries/cuda12
  Copied 0 libraries to /usr/local/lib/python3.12/dist-packages/llcuda/lib
✅ Binaries installed successfully!


✅ llcuda 2.2.0 installed
✅ cuDF 25.06.00
✅ cuGraph 25.06.00
✅ Graphistry 0.50.4
CPU times: user 44.4 s, sys: 10.2 s, total: 54.6 s
Wall time: 1min 22s


## Step 3: Download GGUF Model

In [3]:
%%time
from huggingface_hub import hf_hub_download
import os

# Download a model that fits on single GPU (leaving GPU 1 free)
MODEL_REPO = "unsloth/gemma-3-4b-it-GGUF"
MODEL_FILE = "gemma-3-4b-it-Q4_K_M.gguf"

print(f"📥 Downloading {MODEL_FILE}...")
print(f"   This will run on GPU 0 only.")

model_path = hf_hub_download(
    repo_id=MODEL_REPO,
    filename=MODEL_FILE,
    local_dir="/kaggle/working/models"
)

size_gb = os.path.getsize(model_path) / (1024**3)
print(f"\n✅ Model downloaded: {model_path}")
print(f"   Size: {size_gb:.2f} GB")

📥 Downloading gemma-3-4b-it-Q4_K_M.gguf...
   This will run on GPU 0 only.


gemma-3-4b-it-Q4_K_M.gguf:   0%|          | 0.00/2.49G [00:00<?, ?B/s]


✅ Model downloaded: /kaggle/working/models/gemma-3-4b-it-Q4_K_M.gguf
   Size: 2.32 GB
CPU times: user 5.29 s, sys: 8.2 s, total: 13.5 s
Wall time: 5.14 s


In [9]:
%%time
from huggingface_hub import hf_hub_download
import os

# Try a Llama 3 GGUF model - some builds include FlashAttention
MODEL_REPO = "bartowski/Llama-3.2-3B-Instruct-GGUF"
MODEL_FILE = "Llama-3.2-3B-Instruct-Q4_K_M.gguf"

print(f"📥 Downloading {MODEL_FILE}...")
print(f"   Note: Check if your llcuda binary supports FlashAttention")

model_path = hf_hub_download(
    repo_id=MODEL_REPO,
    filename=MODEL_FILE,
    local_dir="/kaggle/working/models"
)

size_gb = os.path.getsize(model_path) / (1024**3)
print(f"\n✅ Model downloaded: {model_path}")
print(f"   Size: {size_gb:.2f} GB")
print(f"   Note: You'll need to verify FlashAttention support in llcuda")

📥 Downloading Llama-3.2-3B-Instruct-Q4_K_M.gguf...
   Note: Check if your llcuda binary supports FlashAttention


Llama-3.2-3B-Instruct-Q4_K_M.gguf:   0%|          | 0.00/2.02G [00:00<?, ?B/s]


✅ Model downloaded: /kaggle/working/models/Llama-3.2-3B-Instruct-Q4_K_M.gguf
   Size: 1.88 GB
   Note: You'll need to verify FlashAttention support in llcuda
CPU times: user 4.57 s, sys: 7.4 s, total: 12 s
Wall time: 2min 44s


## Step 4: Start llama-server on GPU 0 Only

In [10]:
from llcuda.server import ServerManager

print("="*70)
print("🚀 STARTING LLAMA-SERVER ON GPU 0")
print("="*70)

# Configuration for GPU 0 ONLY (leave GPU 1 for RAPIDS)
print("\n📋 Configuration:")
print("   GPU 0: 100% (llama-server)")
print("   GPU 1: 0% (reserved for RAPIDS)")

server = ServerManager()
server.start_server(
    model_path=model_path,
    host="127.0.0.1",
    port=8090,
    
    # GPU 0 only configuration
    gpu_layers=99,
    tensor_split="1.0,0.0",  # 100% on GPU 0, 0% on GPU 1
    
    # Optimize for single GPU
    ctx_size=4096,
    # Remove or set to False: flash_attention=False,
)

if server.check_server_health():
    print("\n✅ llama-server running on GPU 0!")
else:
    print("\n❌ Server failed to start")

🚀 STARTING LLAMA-SERVER ON GPU 0

📋 Configuration:
   GPU 0: 100% (llama-server)
   GPU 1: 0% (reserved for RAPIDS)
GPU Check:
  Platform: kaggle
  GPU: Tesla T4
  Compute Capability: 7.5
  Status: ✓ Compatible
Starting llama-server...
  Executable: /usr/local/lib/python3.12/dist-packages/llcuda/binaries/cuda12/llama-server
  Model: Llama-3.2-3B-Instruct-Q4_K_M.gguf
  GPU Layers: 99
  Context Size: 4096
  Server URL: http://127.0.0.1:8090
Waiting for server to be ready...... ✓ Ready in 3.0s

✅ llama-server running on GPU 0!


## Step 5: Verify GPU Split

In [11]:
print("="*70)
print("📊 GPU MEMORY SPLIT VERIFICATION")
print("="*70)

!nvidia-smi --query-gpu=index,name,memory.used,memory.free --format=csv

import subprocess
result = subprocess.run(
    ["nvidia-smi", "--query-gpu=index,memory.free", "--format=csv,noheader,nounits"],
    capture_output=True, text=True
)

lines = result.stdout.strip().split('\n')
if len(lines) >= 2:
    gpu1_free = int(lines[1].split(',')[1].strip())
    print(f"\n✅ GPU 1 has {gpu1_free} MiB free for RAPIDS!")

📊 GPU MEMORY SPLIT VERIFICATION
index, name, memory.used [MiB], memory.free [MiB]
0, Tesla T4, 2563 MiB, 12533 MiB
1, Tesla T4, 103 MiB, 14993 MiB

✅ GPU 1 has 14993 MiB free for RAPIDS!


## Step 6: Initialize RAPIDS on GPU 1

In [12]:
import os
# Force RAPIDS to use GPU 1
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

print("="*70)
print("🔥 INITIALIZING RAPIDS ON GPU 1")
print("="*70)

import cudf
import cupy as cp

# Verify we're on the right GPU
print(f"\n📊 RAPIDS GPU Info:")
device = cp.cuda.Device(0)  # Device 0 in filtered view = actual GPU 1
print(f"   Device: {device.id} (filtered view)")
print(f"   Actual GPU: 1 (Tesla T4)")

# Test cuDF on GPU 1
test_df = cudf.DataFrame({
    'source': [0, 1, 2, 3, 4],
    'target': [1, 2, 3, 4, 0],
    'weight': [1.0, 2.0, 1.5, 0.5, 3.0]
})

print(f"\n✅ cuDF working on GPU 1")
print(f"   Test DataFrame: {test_df.shape}")

🔥 INITIALIZING RAPIDS ON GPU 1

📊 RAPIDS GPU Info:
   Device: 0 (filtered view)
   Actual GPU: 1 (Tesla T4)

✅ cuDF working on GPU 1
   Test DataFrame: (5, 3)


## Step 7: Create Sample Graph Data

In [13]:
import cudf
import cugraph

print("="*70)
print("📊 CREATING SAMPLE GRAPH ON GPU 1")
print("="*70)

# Create a sample social network graph
edges = cudf.DataFrame({
    'source': [0, 0, 0, 1, 1, 2, 2, 3, 3, 4, 5, 5, 6, 7, 8, 9],
    'target': [1, 2, 3, 2, 4, 3, 5, 4, 6, 5, 6, 7, 7, 8, 9, 0],
})

# Node labels
node_names = {
    0: "Alice", 1: "Bob", 2: "Charlie", 3: "Diana",
    4: "Eve", 5: "Frank", 6: "Grace", 7: "Henry",
    8: "Ivy", 9: "Jack"
}

print(f"\n📊 Graph created:")
print(f"   Nodes: {len(node_names)}")
print(f"   Edges: {len(edges)}")

# Create cuGraph graph
G = cugraph.Graph()
G.from_cudf_edgelist(edges, source='source', destination='target')

print(f"\n✅ cuGraph graph created on GPU 1")

📊 CREATING SAMPLE GRAPH ON GPU 1

📊 Graph created:
   Nodes: 10
   Edges: 16

✅ cuGraph graph created on GPU 1


## Step 8: Run Graph Analytics on GPU 1

In [14]:
print("="*70)
print("🔬 GPU-ACCELERATED GRAPH ANALYTICS")
print("="*70)

# PageRank
print("\n📊 PageRank Analysis:")
pagerank = cugraph.pagerank(G)
pagerank = pagerank.sort_values('pagerank', ascending=False)

for _, row in pagerank.to_pandas().head(5).iterrows():
    node_id = int(row['vertex'])
    score = row['pagerank']
    name = node_names.get(node_id, f"Node {node_id}")
    print(f"   {name}: {score:.4f}")

# Betweenness Centrality
print("\n📊 Betweenness Centrality:")
bc = cugraph.betweenness_centrality(G)
bc = bc.sort_values('betweenness_centrality', ascending=False)

for _, row in bc.to_pandas().head(5).iterrows():
    node_id = int(row['vertex'])
    score = row['betweenness_centrality']
    name = node_names.get(node_id, f"Node {node_id}")
    print(f"   {name}: {score:.4f}")

print("\n✅ Graph analytics computed on GPU 1")

🔬 GPU-ACCELERATED GRAPH ANALYTICS

📊 PageRank Analysis:
   Alice: 0.1219
   Frank: 0.1204
   Diana: 0.1185
   Charlie: 0.1177
   Henry: 0.0984

📊 Betweenness Centrality:


/usr/local/lib/python3.12/dist-packages/cugraph/link_analysis/pagerank.py:232: UserWarning: Pagerank expects the 'store_transposed' flag to be set to 'True' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWarning)


   Alice: 0.2093
   Frank: 0.1824
   Henry: 0.1389
   Diana: 0.1324
   Charlie: 0.1083

✅ Graph analytics computed on GPU 1


## Step 9: Use LLM to Analyze Graph Results

In [18]:
# Step 9: Use LLM to Analyze Graph Results with llcuda
from llcuda.api.client import LlamaCppClient

print("="*70)
print("🤔 LLM ANALYSIS OF GRAPH RESULTS")
print("="*70)

# Get top PageRank nodes
top_nodes = pagerank.to_pandas().head(3)
top_names = [node_names[int(row['vertex'])] for _, row in top_nodes.iterrows()]

# Create prompt for LLM
prompt = f"""I have a social network graph with 10 people. 
The PageRank analysis shows the most influential people are: {', '.join(top_names)}.

Based on this, what insights can you provide about the network structure? 
Keep your response to 3-4 sentences."""

# Connect to llama-server running on port 8090
client = LlamaCppClient(base_url="http://127.0.0.1:8090")

response = client.chat.create(
    messages=[{"role": "user", "content": prompt}],
    max_tokens=200,
    temperature=0.7
)

print(f"\n📋 LLM Analysis (GPU 0):")
print(response.choices[0].message.content)

print("\n✅ Simultaneous GPU operation:")
print("   GPU 0: LLM inference")
print("   GPU 1: Graph analytics (previously computed)")

🤔 LLM ANALYSIS OF GRAPH RESULTS

📋 LLM Analysis (GPU 0):
{"name": "analyzing_network", "parameters": {"Alice": 1, "Frank": 1, "Diana": 1, "num_nodes": 10}}

✅ Simultaneous GPU operation:
   GPU 0: LLM inference
   GPU 1: Graph analytics (previously computed)


## Step 10: Graphistry Visualization Setup

In [19]:
import graphistry
import pandas as pd

print("="*70)
print("📊 GRAPHISTRY VISUALIZATION")
print("="*70)

# Convert to pandas for Graphistry (works in-notebook)
edges_pd = edges.to_pandas()
edges_pd['source_name'] = edges_pd['source'].map(node_names)
edges_pd['target_name'] = edges_pd['target'].map(node_names)

# Create nodes DataFrame with metrics
pagerank_pd = pagerank.to_pandas()
bc_pd = bc.to_pandas()

nodes_pd = pd.DataFrame({
    'node_id': list(node_names.keys()),
    'name': list(node_names.values())
})
nodes_pd = nodes_pd.merge(
    pagerank_pd.rename(columns={'vertex': 'node_id'}),
    on='node_id'
)
nodes_pd = nodes_pd.merge(
    bc_pd.rename(columns={'vertex': 'node_id'}),
    on='node_id'
)

print(f"\n📊 Prepared for visualization:")
print(f"   Nodes: {len(nodes_pd)}")
print(f"   Edges: {len(edges_pd)}")

# Note: Graphistry requires registration for full visualization
# For demo purposes, we'll show the prepared data
print(f"\n📋 Node Metrics:")
print(nodes_pd[['name', 'pagerank', 'betweenness_centrality']].to_string(index=False))

📊 GRAPHISTRY VISUALIZATION

📊 Prepared for visualization:
   Nodes: 10
   Edges: 16

📋 Node Metrics:
   name  pagerank  betweenness_centrality
  Alice  0.121906                0.209259
    Bob  0.091935                0.037037
Charlie  0.117719                0.108333
  Diana  0.118467                0.132407
    Eve  0.091814                0.057407
  Frank  0.120436                0.182407
  Grace  0.093641                0.060185
  Henry  0.098380                0.138889
    Ivy  0.073543                0.064815
   Jack  0.072161                0.092593


In [20]:
# Create Graphistry visualization that opens in new tab
g = graphistry.bind(source="source", destination="target", node="node_id")
plotter = g.edges(edges_pd).nodes(nodes_pd)

# This will generate a URL that opens in a new browser tab
url = plotter.plot(render=False)
print(f"\n🔗 Graphistry Visualization URL:")
print(f"   {url}")
print("   Open this URL in a new browser tab to view interactive visualization")


🔗 Graphistry Visualization URL:
   https://hub.graphistry.com/graph/graph.html?dataset=a0968cccc7b4478fb0249f3fce3b9da2&type=arrow&viztoken=b5ddcb69-f73f-4728-aba5-477328ddc545&usertag=40fe0b69-pygraphistry-0.50.4&splashAfter=1768851213&info=true
   Open this URL in a new browser tab to view interactive visualization


## Step 11: Interactive LLM + Graph Workflow

In [21]:
print("="*70)
print("🔄 INTERACTIVE LLM + GRAPH WORKFLOW")
print("="*70)

def analyze_node(node_name):
    """Use LLM to analyze a specific node's network position."""
    node_data = nodes_pd[nodes_pd['name'] == node_name].iloc[0]
    
    prompt = f"""Analyze the network position of {node_name}:
    - PageRank score: {node_data['pagerank']:.4f} (higher = more influential)
    - Betweenness centrality: {node_data['betweenness_centrality']:.4f} (higher = more connections)
    
    What does this tell us about {node_name}'s role in the network? Answer in 2 sentences."""
    
    response = client.chat.create(
        messages=[{"role": "user", "content": prompt}],
        max_tokens=100,
        temperature=0.7
    )
    
    return response.choices[0].message.content

# Analyze top 3 nodes
print("\n🔍 Node Analysis:")
for name in ['Alice', 'Charlie', 'Frank']:
    print(f"\n📌 {name}:")
    analysis = analyze_node(name)
    print(f"   {analysis}")

🔄 INTERACTIVE LLM + GRAPH WORKFLOW

🔍 Node Analysis:

📌 Alice:
   {"name": "analyze_network_position", "parameters": {"pr_score": "0.1219", "betweenness_centrality": "0.2093"}}

📌 Charlie:
   {"name": "analyze_network_position", "parameters": {"pr_score": "0.1177", "betweenness_centrality": "0.1083"}}

📌 Frank:
   {"name": "analyze_network_position", "parameters": {"pr_score": "0.1204", "betweenness_centrality": "0.1824"}}


## Step 12: Monitor Both GPUs

In [22]:
print("="*70)
print("📊 DUAL GPU MONITORING")
print("="*70)

!nvidia-smi

print("\n💡 Split-GPU Operation:")
print("   GPU 0: llama-server (GGUF model loaded)")
print("   GPU 1: RAPIDS memory (cuDF/cuGraph data structures)")

📊 DUAL GPU MONITORING
Mon Jan 19 19:34:14 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   68C    P0             29W /   70W |    2677MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-------------------------

## Step 13: Cleanup

In [23]:
print("🛑 Stopping llama-server...")
server.stop_server()

# Clear RAPIDS memory
import gc
del G, edges, pagerank, bc
gc.collect()

print("\n✅ Resources cleaned up")
print("\n📊 Final GPU Status:")
!nvidia-smi --query-gpu=index,memory.used,memory.free --format=csv

🛑 Stopping llama-server...

✅ Resources cleaned up

📊 Final GPU Status:
index, memory.used [MiB], memory.free [MiB]
0, 107 MiB, 14989 MiB
1, 3 MiB, 15093 MiB


## 📚 Summary

### Split-GPU Architecture:
- **GPU 0**: llama-server with `tensor_split=[1.0, 0.0]`
- **GPU 1**: RAPIDS/cuGraph via `CUDA_VISIBLE_DEVICES="1"`

### Key Integration Points:
1. ✅ LLM for natural language analysis
2. ✅ cuGraph for GPU-accelerated graph algorithms
3. ✅ Graphistry for visualization
4. ✅ Combined insights from both

### Code Pattern:
```python
# GPU 0: llama-server
config = ServerConfig(tensor_split=[1.0, 0.0], main_gpu=0)

# GPU 1: RAPIDS
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import cudf, cugraph  # Uses GPU 1
```

---

**Next:** [07-openai-api-client](07-openai-api-client-llcuda-v2.2.0.ipynb)